# Getting Started with ReactorTwin

**ReactorTwin** is a physics-constrained Neural Differential Equation library for building chemical reactor digital twins.

In this notebook, we will cover the core workflow:

1. **Setting up reactors** -- defining kinetics and reactor geometry
2. **Simulating with scipy** -- generating ground-truth trajectories
3. **Training Neural ODEs** -- learning dynamics from data
4. **Applying physics constraints** -- enforcing physical laws
5. **Using the built-in Trainer** -- convenient training with validation and checkpointing

In [ ]:
import numpy as np
import torch
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

from reactor_twin import (
    ArrheniusKinetics, CSTRReactor, BatchReactor,
    NeuralODE, PositivityConstraint, Trainer, ReactorDataGenerator,
    create_exothermic_cstr, create_van_de_vusse_cstr,
)

np.random.seed(42)
torch.manual_seed(42)

## 1. Creating a CSTR Reactor

A **Continuous Stirred-Tank Reactor (CSTR)** is a standard model in chemical engineering. It assumes
perfect mixing inside the vessel so that the outlet composition equals the interior composition.

We define **Arrhenius kinetics** for a simple first-order reaction A -> B, then wrap them in a
`CSTRReactor` with volume, flow rate, and feed conditions.

In [ ]:
kinetics = ArrheniusKinetics(
    name="A_to_B",
    num_reactions=1,
    params={
        "k0": np.array([1e10]),
        "Ea": np.array([50000.0]),
        "stoich": np.array([[-1, 1]]),
    },
)

reactor = CSTRReactor(
    name="tutorial_cstr",
    num_species=2,
    params={
        "V": 100.0,    # Volume (L)
        "F": 10.0,     # Flow rate (L/min)
        "C_feed": [1.0, 0.0],  # Feed concentrations (mol/L)
        "T_feed": 350.0,       # Feed temperature (K)
    },
    kinetics=kinetics,
    isothermal=True,
)

print(f"Reactor: {reactor}")
print(f"State labels: {reactor.get_state_labels()}")
print(f"Initial state: {reactor.get_initial_state()}")

## 2. Simulating with scipy

Every reactor exposes an `ode_rhs` method that returns dy/dt given the current state.
We can plug this directly into `scipy.integrate.solve_ivp` to generate ground-truth trajectories.

In [ ]:
y0 = reactor.get_initial_state()
t_span = np.linspace(0, 10, 100)

sol = solve_ivp(
    reactor.ode_rhs,
    [t_span[0], t_span[-1]],
    y0,
    t_eval=t_span,
    method="LSODA",
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(sol.t, sol.y[0], 'b-', label='C_A')
axes[0].plot(sol.t, sol.y[1], 'r-', label='C_B')
axes[0].set_xlabel('Time (min)')
axes[0].set_ylabel('Concentration (mol/L)')
axes[0].set_title('CSTR: A -> B')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Also try the pre-built exothermic system
reactor_exo = create_exothermic_cstr(isothermal=False)
y0_exo = reactor_exo.get_initial_state()
sol_exo = solve_ivp(reactor_exo.ode_rhs, [0, 5], y0_exo, t_eval=np.linspace(0, 5, 200), method="LSODA")

axes[1].plot(sol_exo.t, sol_exo.y[2], 'g-')
axes[1].set_xlabel('Time (min)')
axes[1].set_ylabel('Temperature (K)')
axes[1].set_title('Non-isothermal CSTR Temperature')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Training a Neural ODE

**Neural ODEs** learn dynamics from data by parameterizing the right-hand side of an ODE with a
neural network:

$$\frac{dy}{dt} = f_\theta(t, y)$$

The network $f_\theta$ is trained to minimize the difference between predicted and observed
trajectories. ReactorTwin provides a `NeuralODE` class that handles integration, loss computation,
and adjoint sensitivity methods.

In [ ]:
# Convert to PyTorch tensors
z0 = torch.tensor(y0, dtype=torch.float32).unsqueeze(0)  # (1, state_dim)
t_tensor = torch.tensor(t_span, dtype=torch.float32)
targets = torch.tensor(sol.y.T, dtype=torch.float32).unsqueeze(0)  # (1, T, state_dim)

# Create Neural ODE
model = NeuralODE(
    state_dim=2,
    hidden_dim=64,
    num_layers=3,
    solver="rk4",
    adjoint=False,
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = []

model.train()
for epoch in range(300):
    optimizer.zero_grad()
    preds = model(z0, t_tensor)
    loss_dict = model.compute_loss(preds, targets)
    loss_dict["total"].backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    losses.append(loss_dict["total"].item())
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:4d}: loss = {losses[-1]:.6f}")

plt.figure(figsize=(8, 3))
plt.semilogy(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Evaluating Predictions

Let's compare the Neural ODE predictions against the ground-truth scipy solution.

In [ ]:
model.eval()
with torch.no_grad():
    preds = model(z0, t_tensor)

pred_np = preds[0].numpy()
true_np = sol.y.T

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for i, (name, color) in enumerate([('C_A', 'blue'), ('C_B', 'red')]):
    axes[i].plot(t_span, true_np[:, i], f'{color[0]}--', label='Ground truth', linewidth=2)
    axes[i].plot(t_span, pred_np[:, i], f'{color[0]}-', label='Neural ODE', linewidth=1.5)
    axes[i].set_xlabel('Time (min)')
    axes[i].set_ylabel(f'{name} (mol/L)')
    axes[i].set_title(f'{name}: Prediction vs Truth')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

mse = np.mean((pred_np - true_np) ** 2)
print(f"MSE: {mse:.6f}")

## 5. Applying Physics Constraints

In chemical systems, concentrations must always be non-negative. The `PositivityConstraint`
enforces this requirement either by **projecting** predictions (hard mode) or by adding a
**penalty** to the loss (soft mode).

In [ ]:
constraint = PositivityConstraint(mode="hard", method="softplus")
constrained_preds, violation = constraint(preds)

print(f"Min (unconstrained): {preds.min().item():.6f}")
print(f"Min (constrained):   {constrained_preds.min().item():.6f}")
print(f"Violation:           {violation.item():.6f}")

## 6. Using the Trainer

ReactorTwin ships with a built-in `Trainer` that handles the training loop, validation splits,
learning rate scheduling, and checkpointing. Combined with `ReactorDataGenerator`, you can go
from reactor definition to trained model in just a few lines.

In [ ]:
# Generate data with the built-in data generator
data_gen = ReactorDataGenerator(reactor=reactor)
z0_batch, t_batch, targets_batch = data_gen.generate(
    num_trajectories=5,
    t_span=(0, 10),
    num_points=50,
    noise_std=0.01,
)

print(f"Generated: z0={z0_batch.shape}, t={t_batch.shape}, targets={targets_batch.shape}")

# Use the Trainer
trainer = Trainer(model=model, device="cpu")
history = trainer.train(
    train_data=(z0_batch, t_batch, targets_batch),
    num_epochs=50,
    lr=5e-4,
    verbose=False,
)
print(f"Final training loss: {history['train_loss'][-1]:.6f}")

## Summary

In this notebook we covered the end-to-end ReactorTwin workflow:

- **Reactor creation** -- `ArrheniusKinetics` + `CSTRReactor` (or the convenience constructors like `create_exothermic_cstr`)
- **Simulation** -- plugging `ode_rhs` into scipy's `solve_ivp`
- **Neural ODE training** -- learning dynamics from trajectory data
- **Physics constraints** -- enforcing positivity with hard/soft modes
- **Built-in Trainer** -- streamlined training with `Trainer` and `ReactorDataGenerator`

Next, explore the full suite of physics constraints in **Notebook 02: Physics Constraints in ReactorTwin**.